In [20]:
import pandas as pd
import numpy as np
import re
import nltk
from collections import Counter
import itertools 
import csv
import altair as alt

In [3]:
df_cty = pd.read_csv('df_cty_final.csv')
df_rock = pd.read_csv('df_rock_final.csv')
df_rb = pd.read_csv('df_rb_final.csv')
df_rb = df_rb[1:]

In [4]:
def get_structures(line):
    WORD_RE = re.compile("\[(.*?)\]")
    token = WORD_RE.findall(line)
    tokens = [w.lower() for w in token if w.lower()]
    return tokens

In [5]:
# Replace coro and verso with chros and verse in future

In [6]:
def get_clean(df):
    structures = []
    for i in df.lyrics:
        structures.append(get_structures(i))

    structures_refinded = []
    for struct in structures:
        temp = []
        for word in struct:
            new = word.split(':',1)[0]
            new = ''.join(i for i in new if not i.isdigit()).strip()

            temp.append(new)
        structures_refinded.append(temp)
        
    struct_values = pd.DataFrame([' '.join(i) for i in  structures_refinded], columns = ['structure'])
    final = pd.DataFrame(struct_values.value_counts(), columns = ['song_counts']).reset_index()
    final = final[final.structure != '']
    
    return final.head(10)

In [7]:
def get_df_for_ngrams(df):
    #global value
    list_of_dicts = []
    for lyrics in df.lyrics:    
        dicts = {}
        for i in lyrics.splitlines():
            if bool(re.match("\[(.*?)\]",i)):
                value = i[1:-1]
                value = value.split(':',1)[0]
                value = ''.join(i for i in value).strip()
            else:
                if value not in dicts:
                    dicts[value] = [i]
                else:
                    dicts[value].append(i)

        list_of_dicts.append(dicts)
    return pd.DataFrame(list_of_dicts)

In [50]:
df_for_cty_ngrams = get_df_for_ngrams(df_cty)
df_for_rock_ngrams = get_df_for_ngrams(df_rock)
df_for_rb_ngrams = get_df_for_ngrams(df_rb)

In [51]:
df_for_cty_ngrams.to_csv('df_for_cty_ngrams.csv', index = False)
df_for_rock_ngrams.to_csv('df_for_rock_ngrams.csv', index = False)
df_for_rb_ngrams.to_csv('df_for_rb_ngrams', index = False)

In [74]:
rock = alt.Chart(get_clean(df_rock),title = 'Top Rock Song Structures').mark_bar(size = 10, opacity = .5,color = '#a24857').encode(
    x= alt.X('song_counts:Q'),
    y= alt.Y('structure:N', sort='-x', title = None , axis = alt.Axis(labelLimit = 500) )
).properties(width = 150, height = 250)

In [75]:
rock

alt.Chart(...)

In [84]:
rock = alt.Chart(get_clean(df_rock),title = 'Top Rock Song Structures').mark_bar(size = 13, opacity = .5,color = '#a24857').encode(
    x= alt.X('song_counts:Q'),
    y= alt.Y('structure:N', sort='-x', title = None , axis = alt.Axis(labelLimit = 500,labelFontSize = 13) )
).properties(width = 150, height = 300)

rb = alt.Chart(get_clean(df_rb),title = 'Top Hip-Hop/Rap Song Structures').mark_bar(size = 13, opacity = .5).encode(
    x= alt.X('song_counts:Q'),
    y= alt.Y('structure:N', sort='-x', title = None , axis = alt.Axis(labelLimit = 500,labelFontSize = 13) )
).properties(width = 150, height = 300)

cty = alt.Chart(get_clean(df_cty),title = 'Top Country Song Structures').mark_bar(size = 13, opacity = .5,color = '#D2B48C').encode(
    x= alt.X('song_counts:Q'),
    y= alt.Y('structure:N', sort='-x', title = None, axis = alt.Axis(labelLimit = 500,labelFontSize = 13)  )
).properties(width = 150, height = 300)

In [85]:
alt.hconcat(rock, rb, cty)

alt.HConcatChart(...)